In [25]:
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import torch

In [26]:
with open("Campus_Pal.json", "r", encoding="utf-8") as file:
    qa_data = json.load(file)

In [27]:
formatted_data = [{"input_text": f"question: {item['question']}", "target_text": item["answer"]} for item in qa_data]

In [28]:
dataset = Dataset.from_list(formatted_data)

In [29]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move model to device
model.to(device)

Using device: cuda


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [30]:
def preprocess_data(examples):
    inputs = tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=256)
    targets = tokenizer(examples["target_text"], padding="max_length", truncation=True, max_length=256)
    inputs["labels"] = targets["input_ids"]
    return inputs

In [31]:
tokenized_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/9357 [00:00<?, ? examples/s]

In [32]:
training_args = TrainingArguments(
    output_dir="./t5_finetuned",
    per_device_train_batch_size=8,
    num_train_epochs=20,
    logging_dir="./logs",
    save_strategy="epoch",
)


In [33]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)
trainer.train()

Step,Training Loss
500,0.676600
1000,0.319000
1500,0.288500
2000,0.284400
2500,0.280100
3000,0.264300
3500,0.264400
4000,0.258400
4500,0.253900
5000,0.248200


TrainOutput(global_step=23400, training_loss=0.2383839380639231, metrics={'train_runtime': 7482.5427, 'train_samples_per_second': 25.01, 'train_steps_per_second': 3.127, 'total_flos': 1.266393236373504e+16, 'train_loss': 0.2383839380639231, 'epoch': 20.0})

In [34]:
model.save_pretrained("./t5_finetuned")
tokenizer.save_pretrained("./t5_finetuned")

('./t5_finetuned\\tokenizer_config.json',
 './t5_finetuned\\special_tokens_map.json',
 './t5_finetuned\\spiece.model',
 './t5_finetuned\\added_tokens.json')

In [ ]:
def generate_answer(query):
    inputs = tokenizer(f"question: {query}", return_tensors="pt", max_length=256, truncation=True)
    output = model.generate(**inputs)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def generate_answer_para(query):
    inputs = tokenizer(f"question: {query}", return_tensors="pt", max_length=1024, truncation=True)
    output = model.generate(**inputs)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def generate_answer_large(query):
    inputs = tokenizer(f"question: {query}", return_tensors="pt", max_length=2048, truncation=True)
    output = model.generate(**inputs)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
query1 = "Who is the founder and Chancellor of Amrita Vishwa Vidyapeetham?"
print(generate_answer(query1))

Sri Mata Amritanandamayi Devi, affectionately known as AMMA, is the founder and Chancellor of the university.


In [ ]:
query2 = "How many campuses does Amrita Vishwa Vidyapeetham have, and can you list them?"
print(generate_answer(query2))

The university has multiple campuses, including Amritapuri, Amaravati, Bengaluru, Chennai, Coimbatore, Kochi, Mysuru, and Faridabad.


In [ ]:
query3 = "What is the size of the Amritapuri campus at Amrita Vishwa Vidyapeetham?"
print(generate_answer(query3))

The campus spans over 80 acres of land


In [ ]:
query4 = "What is the tuition fee per semester for the B.B.A. (Business Analytics) program for the academic year 2025-26?"
print(generate_answer(query4))

Rs. 45,900


In [ ]:
query5 = "What is the NAAC accreditation grade for Amrita Vishwa Vidyapeetham?"
print(generate_answer(query5))

Amrita University has received the topmost 'A++' Grade.


In [ ]:
query1 = "Describe the significance of Amrita's inclusion in the Times Higher Education Impact Rankings."
print(generate_answer_para(query1))

Amrita Vishwa Vidyapeetham's inclusion in the Times Higher Education Impact Rankings marks a significant achievement, reaffirming the university’s dedication to academic excellence and societal impact. As the only Indian university in the World’s Top 100, Amrita's recognition reflects its strong performance in research, teaching, and community engagement. The rankings, which evaluate institutions based on their contributions to real-world challenges, highlight Amrita’s efforts in producing high-quality research that benefits society, industries, and the environment.
Securing the 81st position globally in the 2024 rankings, Amrita has maintained its status as India’s top-ranked university for three consecutive years. This recognition underscores the institution’s commitment to delivering exceptional education through expert faculty and an engaging learning environment. Additionally, Amrita’s strong collaborations with industry and government enhance its research capabilities, providing 

In [ ]:
query2 = "Explain the role and objectives of the Amrita Center for Advanced Research in Ayurveda (ACARA)."
print(generate_answer_para(query2))

ĀCĀRA is a research center dedicated to advancing Ayurveda by integrating traditional wisdom with modern scientific methodologies. Through innovative research, the center aims to validate and enhance Ayurvedic practices, contributing to global healthcare solutions. By bridging ancient knowledge with contemporary science, ĀCĀRA strives to develop evidence-based Ayurvedic therapies.
Located at the Amritapuri campus, the Amrita Centre for Advanced Research in Ayurveda (ACARA) is led by Dr. P. Ram Manohar. Its multidisciplinary team, including Ayurvedic practitioners, biologists, chemists, and pharmacologists, collaborates to explore the pharmacological properties of Ayurvedic herbs and formulations. Their research focuses on identifying active compounds and understanding their mechanisms, which can contribute to the development of new drugs and therapeutic approaches.


In [ ]:
query3 = "What are the main research areas of Amrita Vishwa Vidyapeetham?"
print(generate_answer_para(query3))

Amrita Vishwa Vidyapeetham, a leading educational institution, is dedicated to advancing knowledge and fostering innovation across diverse research domains. Key areas of focus include artificial intelligence and machine learning, where researchers develop intelligent systems to enhance industries and improve lives. The institution also prioritizes cybersecurity, working to safeguard digital systems and protect sensitive data. In healthcare, researchers strive to improve patient care and develop advanced medical technologies. Additionally, Amrita is committed to sustainable technologies, energy, and environmental studies, addressing global challenges like climate change and resource conservation. Through these research efforts, the institution aims to drive technological advancements and contribute to societal progress.


In [ ]:
query4 = "How does Amrita Vishwa Vidyapeetham support rural development and women empowerment initiatives?"
print(generate_answer_para(query4))

Amrita Vishwa Vidyapeetham actively supports rural development and women's empowerment through education, skill training, healthcare, and advocacy. Over 6,000 individuals have been trained through its empowerment programs, and more than 60 workshops have been conducted in schools to promote education in underserved areas. Through initiatives like AmritaCREATE, the university develops educational technologies to bridge learning gaps in rural communities. Vocational training equips women with skills for sustainable livelihoods, fostering financial independence. Additionally, health camps and awareness programs ensure access to essential healthcare services. Through research and advocacy, the institution advances gender equality and social justice, encouraging policies that support women's rights. By integrating education, skills, healthcare, and advocacy, Amrita Vishwa Vidyapeetham empowers women and strengthens rural communities, driving sustainable social progress.


In [ ]:
query5 = "Outline the facilities and resources provided at the Kochi campus for education and research"
print(generate_answer_para(query5))

The Kochi campus is equipped with state-of-the-art facilities to foster a dynamic educational and research environment. The Kochi campus of Amrita Vishwa Vidyapeetham is home to the renowned Amrita Institute of Medical Sciences, a premier multispecialty hospital. It offers diverse academic programs across Medicine, Dentistry, Nursing, Pharmacy, Allied Health Sciences, Nano Sciences, Arts, Humanities & Commerce, Physical Sciences, Computing, and Business. Recognized academic schools include the School of Arts, Humanities & Commerce, School of Physical Sciences, and School of Computing. The campus features modern amenities such as laboratories, smart classrooms, a well-stocked library, a large auditorium, conference rooms, and seminar halls, alongside a traditional Banyan tree that enhances the learning environment. Student support includes a counselling cell, a complaints cell, women's welfare initiatives, medical services, secure hostels, and transport facilities. With organic farming,

In [ ]:
query1 = "Analyse how Amrita's emphasis on research-driven education helps students prepare for global challenges, citing examples from any specific campus."
print(generate_answer_large(query1))

Amrita University’s research-driven education equips students with the skills needed to tackle global challenges. By integrating research into the curriculum, students stay updated on the latest developments, fostering innovation and critical thinking. A key example is the Amrita Center for Sustainable Development, which focuses on renewable energy, waste management, and sustainable agriculture. Students contribute to projects like a solar-powered water purification system for rural India, addressing clean water access and demonstrating the potential of renewable energy solutions.
The Ph.D. in Nanomedicine, Molecular Medicine, and Renewable Energy Related Areas is offered by the Center for Nanosciences at Amrita Vishwa Vidyapeetham, Kochi. The program focuses on advanced research in the areas of nanomedicine, molecular medicine, and renewable energy technologies, combining interdisciplinary approaches to address global challenges in health, medicine, and energy sustainability. The scho

In [ ]:
query2 = "Pharm. D. students undergo a one-year residency in hospital settings. How does this component equip them for leadership roles in clinical pharmacy?"
print(generate_answer_large(query2))

The one-year residency program in hospital settings is a crucial component of Pharm. D. education, preparing students for leadership roles in clinical pharmacy. It offers a comprehensive understanding of the healthcare system, patient care, and medication management. During the residency, students encounter a wide range of clinical scenarios that help them develop essential skills in critical thinking, problem-solving, and decision-making. They also learn to collaborate with healthcare professionals, fostering teamwork and improving patient outcomes.
The residency provides exposure to the complexities of medication management, helping students navigate the healthcare system and develop strategies to improve medication adherence. This knowledge is vital for pharmacists in leadership roles, enabling them to advocate for patients and optimize medication therapy in collaboration with other professionals. Additionally, the program enhances communication and interpersonal skills, teaching st

In [ ]:
query3 = "If a student skilled in Python, Java, and data analysis is looking to specialize, which programs at Amrita Vishwa Vidyapeetham would you recommend, and why?"
print(generate_answer_large(query3))

For a student skilled in Python, Java, and data analysis, Amrita Vishwa Vidyapeetham offers a variety of advanced programs that align with their skills and interests. The university’s Master of Science programs in Data Science, Artificial Intelligence, Computer Vision, Bioinformatics, and Cybersecurity provide in-depth knowledge and practical experience in cutting-edge technologies. These programs cover critical areas such as machine learning, deep learning, natural language processing, image processing, genomics, and cryptography. Additionally, students are exposed to real-world case studies and hands-on projects, enabling them to gain practical insights and problem-solving skills that are highly valued in the industry.
The focus on interdisciplinary learning at Amrita enhances students’ abilities to tackle complex challenges across various domains. Programs like Data Science and AI incorporate big data analytics and statistical techniques, while Computer Vision and Bioinformatics del

In [ ]:
query4 = "If a student is passionate about renewable energy and sustainable solutions, how could Amrita Vishwa Vidyapeetham’s academic and research initiatives support their ambitions?"
print(generate_answer_large(query4))

Amrita Vishwa Vidyapeetham offers extensive academic and research support for students passionate about renewable energy and sustainable solutions. The university's dedicated research centers, such as the Centre for Energy Research and the Centre for Sustainable Development, provide students with the opportunity to engage in cutting-edge research and collaborate with industry partners, government agencies, and other academic institutions. These centers foster innovation in renewable energy technologies, addressing real-world challenges and promoting sustainable development. Additionally, the university's advanced laboratories and simulation tools enable students to conduct experiments and develop prototypes, providing essential resources like solar simulators and wind tunnels for hands-on research.
The university also encourages interdisciplinary collaboration, allowing students from various disciplines like engineering, science, and business to work together to develop comprehensive s

In [ ]:
query5 = "Analyse the challenges Amrita may face in organizing international conferences and propose solutions to mitigate them."
print(generate_answer_large(query5))

Amrita, as an organizer of international conferences, faces logistical, cultural, and technological challenges in ensuring successful events. Logistical issues include venue selection, accommodation, travel arrangements, and scheduling. Coordinating these aspects for participants from various countries and time zones can be complex. To address these challenges, Amrita can collaborate with local event organizers and tourism boards to identify suitable venues, negotiate group discounts, and streamline the accommodation process. Additionally, partnering with travel agencies and providing a comprehensive travel guide can help manage travel logistics. Scheduling can be optimized using project management tools to create a detailed conference program that accommodates speakers' availability and attendees' preferences.
Cultural challenges also play a significant role in organizing international conferences. Language barriers and cultural differences can hinder effective communication and creat